In [ ]:
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from bintree import (
    ForwardBinomialTree,
    call_payoff,
    put_payoff,
    digital_call_payoff
)

In [ ]:
# Download available option maturities for a stock ticker
ticker = "NVDA"
stock = yf.Ticker(ticker)

# Get available expiration dates
expirations = stock.options
print(f"Available option maturities for {ticker}:")
for exp in expirations:
    print(f"  {exp}")

In [ ]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

# Get current stock price and download timestamp
current_price = stock.info['currentPrice']
download_timestamp = datetime.now(ZoneInfo('America/New_York'))

# Market hours: 9:30 AM - 4:00 PM ET, weekdays only
market_open = download_timestamp.replace(hour=9, minute=30, second=0, microsecond=0)
market_close = download_timestamp.replace(hour=16, minute=0, second=0, microsecond=0)
is_weekday = download_timestamp.weekday() < 5  # Monday=0, Friday=4

# Determine data timestamp based on when we query
if is_weekday and market_open <= download_timestamp <= market_close:
    # During trading hours: data is delayed by 15 minutes
    data_timestamp = download_timestamp - timedelta(minutes=15)
    timestamp_note = "During market hours - 15 min delay applied"
else:
    # After hours / weekend: data reflects market close
    if download_timestamp < market_open and is_weekday:
        # Before market open on a weekday - use previous trading day's close
        days_back = 1 if download_timestamp.weekday() != 0 else 3  # Monday -> Friday
        data_timestamp = (download_timestamp - timedelta(days=days_back)).replace(
            hour=16, minute=0, second=0, microsecond=0)
        timestamp_note = "Before market open - using previous close"
    elif not is_weekday:
        # Weekend - use Friday's close
        days_back = download_timestamp.weekday() - 4  # Days since Friday
        data_timestamp = (download_timestamp - timedelta(days=days_back)).replace(
            hour=16, minute=0, second=0, microsecond=0)
        timestamp_note = "Weekend - using Friday's close"
    else:
        # After market close on a weekday
        data_timestamp = market_close
        timestamp_note = "After market close - using today's close"

print(f"Current price of {ticker}: ${current_price:.2f}")
print(f"Download time (ET): {download_timestamp.strftime('%Y-%m-%d %H:%M:%S %Z')}")
print(f"Data timestamp (ET): {data_timestamp.strftime('%Y-%m-%d %H:%M:%S %Z')}")
print(f"Note: {timestamp_note}")

# Select an expiration date
selected_expiration = expirations[4]  # Change index to select different expiration
print(f"\nSelected expiration: {selected_expiration}")

# Download option chain for selected expiration
option_chain = stock.option_chain(selected_expiration)

# Option chain contains calls and puts DataFrames
calls = option_chain.calls
puts = option_chain.puts

print(f"\nCalls ({len(calls)} contracts):")
display(calls.head())

print(f"\nPuts ({len(puts)} contracts):")
display(puts.head())

In [ ]:
# Options expire at 4pm ET on the expiration date
expiration_date = datetime.strptime(selected_expiration, '%Y-%m-%d')
expiration_datetime_et = expiration_date.replace(hour=16, minute=0, second=0, tzinfo=ZoneInfo('America/New_York'))

# data_timestamp is already in ET from previous cell

# Calculate time to expiration in years
time_to_expiration = expiration_datetime_et - data_timestamp
T_years = time_to_expiration.total_seconds() / (365.25 * 24 * 60 * 60)

print(f"Expiration: {expiration_datetime_et.strftime('%Y-%m-%d %H:%M %Z')}")
print(f"Data timestamp: {data_timestamp.strftime('%Y-%m-%d %H:%M:%S %Z')}")
print(f"Time to expiration: {time_to_expiration}")
print(f"Time to expiration (years): {T_years:.6f}")

In [ ]:
# Download last year of dividend data
one_year_ago = data_timestamp - timedelta(days=365)
dividends = stock.dividends

# Filter to last year
dividends_last_year = dividends[dividends.index >= one_year_ago.strftime('%Y-%m-%d')]

print(f"Dividends for {ticker} over the last year:")
print(dividends_last_year)
print(f"\nTotal dividends: ${dividends_last_year.sum():.4f}")
print(f"Number of payments: {len(dividends_last_year)}")

In [ ]:
# Add time-to-expiration to option DataFrames
calls['T'] = T_years
puts['T'] = T_years

# Calculate continuously compounded dividend yield
# Annual dividend / current price gives simple yield
# Continuous yield: q = ln(1 + D/S) ≈ D/S for small yields
annual_dividends = dividends_last_year.sum()
simple_div_yield = annual_dividends / current_price
div_yield = np.log(1 + simple_div_yield)  # Continuously compounded

print(f"Annual dividends: ${annual_dividends:.4f}")
print(f"Current price: ${current_price:.2f}")
print(f"Simple dividend yield: {simple_div_yield:.4%}")
print(f"Continuously compounded dividend yield: {div_yield:.4%}")

# Add dividend yield to option DataFrames
calls['div_yield'] = div_yield
puts['div_yield'] = div_yield

print(f"\nCalls DataFrame columns: {list(calls.columns)}")
print(f"Puts DataFrame columns: {list(puts.columns)}")

In [ ]:
import pandas_datareader.data as web

# Download latest overnight SOFR rate from FRED
start_date = (data_timestamp - timedelta(days=7)).strftime('%Y-%m-%d')
end_date = data_timestamp.strftime('%Y-%m-%d')

sofr_data = web.DataReader('SOFR', 'fred', start_date, end_date)
rf = sofr_data.iloc[-1, 0] / 100  # Convert from percentage

print(f"Latest SOFR rate: {rf:.4%}")
print(f"As of: {sofr_data.index[-1].strftime('%Y-%m-%d')}")

# Add to option DataFrames
calls['rf'] = rf
puts['rf'] = rf

In [ ]:
# Calculate mid prices as average of bid and ask
calls['mid'] = (calls['bid'] + calls['ask']) / 2
puts['mid'] = (puts['bid'] + puts['ask']) / 2

# Plot 1: Standard scale
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(calls['strike'], calls['mid'], 'b.-', label='Calls')
axes[0].plot(puts['strike'], puts['mid'], 'r.-', label='Puts')
axes[0].axvline(current_price, color='gray', linestyle='--', label=f'Current price (${current_price:.2f})')
axes[0].set_xlabel('Strike')
axes[0].set_ylabel('Mid Price')
axes[0].set_title(f'{ticker} Option Mid Prices - {selected_expiration}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Log scale (y-axis)
axes[1].plot(calls['strike'], calls['mid'], 'b.-', label='Calls')
axes[1].plot(puts['strike'], puts['mid'], 'r.-', label='Puts')
axes[1].axvline(current_price, color='gray', linestyle='--', label=f'Current price (${current_price:.2f})')
axes[1].set_xlabel('Strike')
axes[1].set_ylabel('Mid Price (log scale)')
axes[1].set_yscale('log')
axes[1].set_title(f'{ticker} Option Mid Prices (Log Scale) - {selected_expiration}')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Filter for out-of-the-money options
# OTM calls: strike > current price
# OTM puts: strike < current price

otm_calls = calls[calls['strike'] > current_price].copy()
otm_puts = puts[puts['strike'] < current_price].copy()

print(f"Current price: ${current_price:.2f}")
print(f"\nOTM Calls ({len(otm_calls)} contracts) - Strike > ${current_price:.2f}:")
display(otm_calls)

print(f"\nOTM Puts ({len(otm_puts)} contracts) - Strike < ${current_price:.2f}:")
display(otm_puts)

In [ ]:
# Download 12 months of daily price history
end_date_hist = data_timestamp.strftime('%Y-%m-%d')
start_date_hist = (data_timestamp - timedelta(days=365)).strftime('%Y-%m-%d')

hist = stock.history(start=start_date_hist, end=end_date_hist)

# Calculate daily log returns
hist['return'] = hist['Close'] / hist['Close'].shift(1)-1.0

# Estimate volatility (standard deviation of daily returns)
daily_vol = hist['return'].std()

# Annualize volatility (assuming 252 trading days per year)
physical_sigma = daily_vol * np.sqrt(252)

print(f"Historical data: {start_date_hist} to {end_date_hist}")
print(f"Number of trading days: {len(hist)}")
print(f"Daily volatility: {daily_vol:.4%}")
print(f"Annualized volatility (physical_sigma): {physical_sigma:.4%}")

# Add to OTM option tables
otm_calls['physical_sigma'] = physical_sigma
otm_puts['physical_sigma'] = physical_sigma

In [ ]:
# Price OTM options using binomial tree with different number of periods using historical volatility
periods_list = [5, 37, 200]

# Price OTM calls
for N in periods_list:
    prices = []
    for _, row in otm_calls.iterrows():
        tree = ForwardBinomialTree(
            S0=current_price,
            T_maturity=row['T'],
            N_periods=N,
            sigma=row['physical_sigma'],
            rf=row['rf'],
            div_yld=row['div_yield']
        )
        price = tree.price_american(call_payoff(row['strike']))
        prices.append(price)
    otm_calls[f'bin_{N}'] = prices

# Price OTM puts
for N in periods_list:
    prices = []
    for _, row in otm_puts.iterrows():
        tree = ForwardBinomialTree(
            S0=current_price,
            T_maturity=row['T'],
            N_periods=N,
            sigma=row['physical_sigma'],
            rf=row['rf'],
            div_yld=row['div_yield']
        )
        price = tree.price_american(put_payoff(row['strike']))
        prices.append(price)
    otm_puts[f'bin_{N}'] = prices

print("OTM Calls with binomial prices:")
display(otm_calls[['strike', 'mid', 'bin_5', 'bin_37', 'bin_200']])

print("\nOTM Puts with binomial prices:")
display(otm_puts[['strike', 'mid', 'bin_5', 'bin_37', 'bin_200']])

In [ ]:
# Plot OTM Calls: observed vs binomial prices
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Calls - Regular scale
axes[0, 0].plot(otm_calls['strike'], otm_calls['mid'], 'b.-', linewidth=2, label='Market mid')
axes[0, 0].plot(otm_calls['strike'], otm_calls['bin_5'], 'c.--', alpha=0.5, label='Binomial N=5')
axes[0, 0].plot(otm_calls['strike'], otm_calls['bin_37'], 'm.--', alpha=0.6, label='Binomial N=37')
axes[0, 0].plot(otm_calls['strike'], otm_calls['bin_200'], 'g.--', alpha=0.7, label='Binomial N=200')
axes[0, 0].set_xlabel('Strike')
axes[0, 0].set_ylabel('Price')
axes[0, 0].set_title(f'OTM Calls - {ticker} {selected_expiration}')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Calls - Log scale
axes[0, 1].plot(otm_calls['strike'], otm_calls['mid'], 'b.-', linewidth=2, label='Market mid')
axes[0, 1].plot(otm_calls['strike'], otm_calls['bin_5'], 'c.--', alpha=0.5, label='Binomial N=5')
axes[0, 1].plot(otm_calls['strike'], otm_calls['bin_37'], 'm.--', alpha=0.6, label='Binomial N=37')
axes[0, 1].plot(otm_calls['strike'], otm_calls['bin_200'], 'g.--', alpha=0.7, label='Binomial N=200')
axes[0, 1].set_xlabel('Strike')
axes[0, 1].set_ylabel('Price (log scale)')
axes[0, 1].set_yscale('log')
axes[0, 1].set_title(f'OTM Calls (Log Scale) - {ticker} {selected_expiration}')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Puts - Regular scale
axes[1, 0].plot(otm_puts['strike'], otm_puts['mid'], 'r.-', linewidth=2, label='Market mid')
axes[1, 0].plot(otm_puts['strike'], otm_puts['bin_5'], 'c.--', alpha=0.5, label='Binomial N=5')
axes[1, 0].plot(otm_puts['strike'], otm_puts['bin_37'], 'm.--', alpha=0.6, label='Binomial N=37')
axes[1, 0].plot(otm_puts['strike'], otm_puts['bin_200'], 'g.--', alpha=0.7, label='Binomial N=200')
axes[1, 0].set_xlabel('Strike')
axes[1, 0].set_ylabel('Price')
axes[1, 0].set_title(f'OTM Puts - {ticker} {selected_expiration}')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Puts - Log scale
axes[1, 1].plot(otm_puts['strike'], otm_puts['mid'], 'r.-', linewidth=2, label='Market mid')
axes[1, 1].plot(otm_puts['strike'], otm_puts['bin_5'], 'c.--', alpha=0.5, label='Binomial N=5')
axes[1, 1].plot(otm_puts['strike'], otm_puts['bin_37'], 'm.--', alpha=0.6, label='Binomial N=37')
axes[1, 1].plot(otm_puts['strike'], otm_puts['bin_200'], 'g.--', alpha=0.7, label='Binomial N=200')
axes[1, 1].set_xlabel('Strike')
axes[1, 1].set_ylabel('Price (log scale)')
axes[1, 1].set_yscale('log')
axes[1, 1].set_title(f'OTM Puts (Log Scale) - {ticker} {selected_expiration}')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
from scipy.optimize import brentq

# Demonstration: Calculate implied volatility for the first OTM call (lowest strike)
option = otm_calls.iloc[0]

print("Option details:")
print(f"  Strike: ${option['strike']:.2f}")
print(f"  Market mid price: ${option['mid']:.4f}")
print(f"  Physical sigma: {option['physical_sigma']:.4%}")
print(f"  T: {option['T']:.4f} years")
print(f"  rf: {option['rf']:.4%}")
print(f"  div_yield: {option['div_yield']:.4%}")

# Define pricing function as a function of sigma
def binomial_price(sigma, S0, K, T, rf, div_yld, N=200):
    tree = ForwardBinomialTree(
        S0=S0,
        T_maturity=T,
        N_periods=N,
        sigma=sigma,
        rf=rf,
        div_yld=div_yld
    )
    return tree.price_american(call_payoff(K))

# Define objective: difference between model price and market price
def price_difference(sigma):
    model_price = binomial_price(
        sigma=sigma,
        S0=current_price,
        K=option['strike'],
        T=option['T'],
        rf=option['rf'],
        div_yld=option['div_yield'],
        N=200
    )
    return model_price - option['mid']

# Solve for implied volatility using Brent's method
# Search between 1% and 500% volatility
try:
    implied_vol = brentq(price_difference, 0.01, 5.0)
    
    # Verify the result
    model_price_at_iv = binomial_price(
        sigma=implied_vol,
        S0=current_price,
        K=option['strike'],
        T=option['T'],
        rf=option['rf'],
        div_yld=option['div_yield'],
        N=200
    )
    
    print(f"\nImplied volatility calculation:")
    print(f"  Implied volatility: {implied_vol:.4%}")
    print(f"  Model price at IV: ${model_price_at_iv:.4f}")
    print(f"  Market mid price: ${option['mid']:.4f}")
    print(f"  Difference: ${abs(model_price_at_iv - option['mid']):.6f}")
except ValueError as e:
    print(f"\nCould not find implied volatility: {e}")

In [ ]:
# Calculate implied volatility for all OTM options

def calc_implied_vol(row, option_type='call', N=200):
    """Calculate implied volatility for a single option."""
    payoff = call_payoff(row['strike']) if option_type == 'call' else put_payoff(row['strike'])
    
    def price_diff(sigma):
        tree = ForwardBinomialTree(
            S0=current_price,
            T_maturity=row['T'],
            N_periods=N,
            sigma=sigma,
            rf=row['rf'],
            div_yld=row['div_yield']
        )
        return tree.price_american(payoff) - row['mid']
    
    try:
        return brentq(price_diff, 0.01, 5.0)
    except ValueError:
        return np.nan

# Calculate IV for OTM calls
print("Calculating implied volatility for OTM calls...")
otm_calls['iv'] = otm_calls.apply(lambda row: calc_implied_vol(row, 'call'), axis=1)

# Calculate IV for OTM puts
print("Calculating implied volatility for OTM puts...")
otm_puts['iv'] = otm_puts.apply(lambda row: calc_implied_vol(row, 'put'), axis=1)

print("\nOTM Calls with implied volatility:")
display(otm_calls[['strike', 'mid', 'physical_sigma', 'iv']])

print("\nOTM Puts with implied volatility:")
display(otm_puts[['strike', 'mid', 'physical_sigma', 'iv']])

In [ ]:
# Plot implied volatility: our calculation vs data source (combined)
fig, ax = plt.subplots(figsize=(12, 6))

# OTM Puts (left side of smile)
ax.plot(otm_puts['strike'], otm_puts['iv'], 'b.-', linewidth=2, label='Our IV (binomial)')
ax.plot(otm_puts['strike'], otm_puts['impliedVolatility'], 'r.--', alpha=0.7, label='Data IV')

# OTM Calls (right side of smile)
ax.plot(otm_calls['strike'], otm_calls['iv'], 'b.-', linewidth=2)
ax.plot(otm_calls['strike'], otm_calls['impliedVolatility'], 'r.--', alpha=0.7)

# Reference lines
ax.axhline(otm_calls['physical_sigma'].iloc[0], color='gray', linestyle=':', label=f'Physical σ ({otm_calls["physical_sigma"].iloc[0]:.2%})')
ax.axvline(current_price, color='green', linestyle='--', alpha=0.5, label=f'Spot (${current_price:.2f})')

ax.set_xlabel('Strike')
ax.set_ylabel('Implied Volatility')
ax.set_title(f'Volatility Smile - {ticker} {selected_expiration}')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Use the implied volatility of the first OTM call to build a tree
# and calculate risk-neutral probabilities of various price outcomes

first_call = otm_calls.iloc[0]
iv = first_call['iv']

print(f"First OTM call option:")
print(f"  Strike: ${first_call['strike']:.2f}")
print(f"  Implied volatility: {iv:.4%}")

# Create 200-step binomial tree with implied volatility
tree = ForwardBinomialTree(
    S0=current_price,
    T_maturity=first_call['T'],
    N_periods=200,
    sigma=iv,
    rf=first_call['rf'],
    div_yld=first_call['div_yield']
)

# Calculate probabilities at each node
tree.calculate_probabilities()

# Get terminal nodes (at maturity)
terminal_period = tree.N_periods
terminal_nodes = tree.tree[terminal_period]

# Extract terminal prices and their probabilities
terminal_prices = []
terminal_probs = []
for index in range(terminal_period + 1):
    node = terminal_nodes[index]
    terminal_prices.append(node.stock_price)
    terminal_probs.append(node.probability)

terminal_prices = np.array(terminal_prices)
terminal_probs = np.array(terminal_probs)

# Price thresholds to analyze
thresholds = [first_call['strike'], 200, 250, 300]

print(f"\nCurrent stock price: ${current_price:.2f}")
print(f"Time to maturity: {first_call['T']:.4f} years")
print(f"\nRisk-neutral probabilities of terminal price outcomes:")
print("=" * 80)

for threshold in thresholds:
    # Probability that price > threshold (upside)
    prob_above = np.sum(terminal_probs[terminal_prices > threshold])
    
    # Simple return and log return
    simple_ret = threshold / current_price - 1
    log_ret = np.log(1 + simple_ret)  # = log(threshold / current_price)
    
    # Symmetric negative log return
    neg_log_ret = -log_ret
    symmetric_simple_ret = np.exp(neg_log_ret) - 1
    symmetric_price = current_price * np.exp(neg_log_ret)
    
    # Probability that price < symmetric_price (downside)
    prob_below = np.sum(terminal_probs[terminal_prices < symmetric_price])
    
    print(f"\nThreshold: ${threshold:.2f}")
    print(f"  Upside:")
    print(f"    Price > ${threshold:.2f}")
    print(f"    Simple return: {simple_ret:+.2%}, Log return: {log_ret:+.4f}")
    print(f"    Probability: {prob_above:.4%}")
    print(f"  Symmetric downside:")
    print(f"    Price < ${symmetric_price:.2f}")
    print(f"    Simple return: {symmetric_simple_ret:+.2%}, Log return: {neg_log_ret:+.4f}")
    print(f"    Probability: {prob_below:.4%}")

# Summary statistics of terminal distribution
print(f"\n" + "=" * 80)
print(f"Terminal distribution summary (risk-neutral):")
expected_price = np.sum(terminal_prices * terminal_probs)
print(f"  Expected terminal price: ${expected_price:.2f}")
print(f"  Min terminal price: ${terminal_prices.min():.2f}")
print(f"  Max terminal price: ${terminal_prices.max():.2f}")

In [ ]:
# Visualize the terminal price distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Price distribution (linear scale)
axes[0].bar(terminal_prices, terminal_probs, width=(terminal_prices.max() - terminal_prices.min()) / len(terminal_prices) * 0.8, alpha=0.7, color='steelblue')
axes[0].axvline(current_price, color='green', linestyle='--', linewidth=2, label=f'Current (${current_price:.2f})')
axes[0].axvline(expected_price, color='orange', linestyle='--', linewidth=2, label=f'Expected (${expected_price:.2f})')
for threshold in thresholds:
    axes[0].axvline(threshold, color='red', linestyle=':', alpha=0.5)
axes[0].set_xlabel('Terminal Stock Price')
axes[0].set_ylabel('Probability')
axes[0].set_title(f'Risk-Neutral Terminal Price Distribution\n{ticker} - IV={iv:.2%}, T={first_call["T"]:.3f}y')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Log-price distribution (shows symmetry/asymmetry)
log_prices = np.log(terminal_prices / current_price)
axes[1].bar(log_prices, terminal_probs, width=(log_prices.max() - log_prices.min()) / len(log_prices) * 0.8, alpha=0.7, color='steelblue')
axes[1].axvline(0, color='green', linestyle='--', linewidth=2, label='Current (log=0)')
axes[1].axvline(np.log(expected_price / current_price), color='orange', linestyle='--', linewidth=2, label=f'Expected ({np.log(expected_price/current_price):+.3f})')
for threshold in thresholds:
    log_thresh = np.log(threshold / current_price)
    axes[1].axvline(log_thresh, color='red', linestyle=':', alpha=0.5)
    axes[1].axvline(-log_thresh, color='blue', linestyle=':', alpha=0.5)
axes[1].set_xlabel('Log Return: ln(S_T / S_0)')
axes[1].set_ylabel('Probability')
axes[1].set_title('Log-Return Distribution\n(Red: upside thresholds, Blue: symmetric downside)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 